In [ ]:
# Import required modules
import socket
import ast
import threading

BUFFER = 2**20
HOST = '127.0.0.1'
# HOST = '172.20.10.2'
PORT = 1234 # You can use any port between 0 to 65535
LISTENER_LIMIT = 5
active_clients = [] # List of all currently connected users
users = {}


# Function to listen for upcoming messages from a client
def listen_for_messages(client, username):

    while 1:

        message = client.recv(BUFFER).decode('utf-8')
        
        # default -> Group
        # format of rcvd msg { ***PRIVATE***username***data }
        # send to one user (PRIVATE)  
        if message != '' and "***PRIVATE***" in message:
            target_client_username  = message.split("***PRIVATE***")[1].split("###")[0]
            
            if target_client_username in users.keys():
                target_client_addr = users.get(target_client_username)
                message = message.split("***PRIVATE***")[1].split("###")[1]
                final_msg = username + '~' + message
                send_message_to_client(target_client_addr, final_msg)
                
            else:
                print("Error -> username not in active client list !!!")
            
            
        # broadcast to group (GROUP)
        elif message != '' :
            final_msg = username + '~' + message
            send_messages_to_all(final_msg)

        else:
            print(f"The message send from client {username} is empty")


# Function to send message to a single client
def send_message_to_client(client, message):

    client.sendall(message.encode())

# Function to send any new message to all the clients that
# are currently connected to this server
def send_messages_to_all(message):
    
    for user in active_clients:

        send_message_to_client(user[1], message)

# Function to handle client
def client_handler(client):
    
    # Server will listen for client message that will
    # Contain the username
    while 1:

        username = client.recv(BUFFER).decode('utf-8')
        if username != '':
            active_clients.append((username, client))
            users.update({username : client})
            prompt_message = "SERVER~" + f"{username} added to the chat"
            send_messages_to_all(prompt_message)
            print(active_clients)
            
            # for sending usernames to clients
            string_users = str(list(users.keys()))
            
            send_messages_to_all("***USERS***" + string_users)
            break
            
        else:
            print("Client username is empty")

    threading.Thread(target=listen_for_messages, args=(client, username, )).start()

# Main function
def main():

    # Creating the socket class object
    # AF_INET: we are going to use IPv4 addresses
    # SOCK_STREAM: we are using TCP packets for communication
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Creating a try catch block
    try:
        # Provide the server with an address in the form of
        # host IP and port
        server.bind((HOST, PORT))
        print(f"Running the server on {HOST} {PORT}")
    except:
        print(f"Unable to bind to host {HOST} and port {PORT}")

    # Set server limit
    server.listen(LISTENER_LIMIT)

    # This while loop will keep listening to client connections
    while 1:

        client, address = server.accept()
        print(f"Successfully connected to client {address[0]} {address[1]}")

        threading.Thread(target=client_handler, args=(client, )).start()


if __name__ == '__main__':
    main()

Running the server on 127.0.0.1 1234
Successfully connected to client 127.0.0.1 50568
[('user1', <socket.socket fd=1792, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 50568)>)]
Successfully connected to client 127.0.0.1 50569
[('user1', <socket.socket fd=1792, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 50568)>), ('user2', <socket.socket fd=2012, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 50569)>)]
